# Process wheel data into movements 

In [1]:
""" 
IMPORTS
"""
import os
import autograd.numpy as np
import pickle
from collections import defaultdict
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from scipy import stats
from datetime import datetime
from scipy.interpolate import interp1d
from numba import njit, prange 
from one.api import ONE
import concurrent.futures

# Get my functions
functions_path =  '/home/ines/repositories/representation_learning_variability/Functions/'
# functions_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/'
os.chdir(functions_path)
from data_processing import process_quiescence
from wheel_functions import wheel_trial_epoch
one = ONE(base_url='https://alyx.internationalbrainlab.org')

/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/one/api.py:1465: UserWarning: Newer cache tables require ONE version 2.7 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


## Load training data

In [2]:
# Choose a session with good QC
data_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/'
# data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Video and wheel/'
# save_path = '/home/ines/repositories/representation_learning_variability/Exported figures/'

os.chdir(data_path)
pass_qc = pickle.load(open(data_path + "good_brainwide_sessions_metadata", "rb"))
pass_qc = pickle.load(open(data_path + "good_dlc_bwm_sessions_metadata", "rb"))
mouse_names = pass_qc['subject_nickname'].unique()

In [3]:
def process_wheel_movements(mouse_name):

    trial_data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Trial_data/'
    trial_data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Trial_data/'
    os.chdir(trial_data_path)

    training_session = 'last'
    threshold = 0.2 # Need to check if this makes sense
    min_period = 400 # This is to match the minimum length of quiescence periods
    bin_size = 0.05
    mouse_data = pickle.load(open('training_data_trials_'+ mouse_name, "rb"))

    if training_session == 'last':
        mouse_session_num = np.max(mouse_data['training_day'])
        use_data = mouse_data.loc[mouse_data['training_day'] >= mouse_session_num]
    else:
        use_data = mouse_data.loc[mouse_data['training_day'] == training_session]

    wheel_data_path =  '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Wheel/'
    wheel_data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Wheel/'
    
    existing_files = os.listdir(wheel_data_path)
    os.chdir(wheel_data_path)

    # Loop through sessions
    mouse_sessions = use_data['session'].unique()
    for s, session in enumerate(mouse_sessions):
        filename = "wheel_movements_" + str(training_session) + '_' + session
        
        if filename not in existing_files:
            session_trials = use_data.loc[use_data['session']==session]
            session_trials['index'] = np.arange(0, len(session_trials))
            
            df = wheel_trial_epoch(one, session_trials, session, bin_size, threshold, min_period).dropna()  

            # Save to a unique file
            pickle.dump(df, open(filename, "wb"))

def parallel_process_data(mouse_names, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, mouse_names)

In [4]:
# Loop through animals
function_name = process_wheel_movements
parallel_process_data(mouse_names, function_name)

/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/UCLA034/2021-12-07/001/alf/_ibl_wheel.position.npy: 100%|██████████| 3.00M/3.00M [00:02<00:00, 1.31MB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/churchlandlab_ucla/Subjects/UCLA034/2021-12-07/001/alf/_ibl_wheel.timestamps.npy: 100%|██████████| 3.00M/3.00M [00:02<00:00, 1.28MB/s]
